# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [2]:
from collections import deque
import heapq
import time
from typing import Set, Tuple, List, Dict, FrozenSet

class JealousHusbandsState:
    def __init__(self, left: FrozenSet[str], island: FrozenSet[str], right: FrozenSet[str], boat: str):
        self.left = left
        self.island = island
        self.right = right
        self.boat = boat  # 'L', 'I', or 'R'
    
    def __eq__(self, other):
        if not isinstance(other, JealousHusbandsState):
            return False
        return (self.left == other.left and 
                self.island == other.island and 
                self.right == other.right and 
                self.boat == other.boat)
    
    def __hash__(self):
        return hash((self.left, self.island, self.right, self.boat))
    
    def __str__(self):
        return f"Left: {sorted(self.left)}, Island: {sorted(self.island)}, Right: {sorted(self.right)}, Boat: {self.boat}"

class JealousHusbandsProblem:
    def __init__(self, num_couples: int):
        self.num_couples = num_couples
        self.all_people = frozenset([f"H{i+1}" for i in range(num_couples)] + 
                                  [f"W{i+1}" for i in range(num_couples)])
        self.initial_state = JealousHusbandsState(
            left=self.all_people,
            island=frozenset(),
            right=frozenset(),
            boat='L'
        )
        self.goal_state_check = lambda state: len(state.right) == len(self.all_people)
    
    def is_valid_state(self, state: JealousHusbandsState) -> bool:
        """Check if a state satisfies the jealousy constraint."""
        for location in [state.left, state.island, state.right]:
            wives = {person for person in location if person.startswith('W')}
            husbands = {person for person in location if person.startswith('H')}
            
            for wife in wives:
                wife_num = int(wife[1:])
                husband = f"H{wife_num}"
                # If wife is present, but her husband is not, and other husbands are present
                if husband not in location and any(h for h in husbands):
                    return False
        
        return True
    
    def get_valid_moves(self, state: JealousHusbandsState) -> List[Tuple[FrozenSet[str], str]]:
        """Get all valid moves from the current state."""
        valid_moves = []
        
        if state.boat == 'L':
            # Boat is at left bank, can move 1 or 2 people to the island
            people_at_location = state.left
            destination = 'I'
        elif state.boat == 'I':
            # Boat is at island, can move 1 or 2 people to either bank
            people_at_location = state.island
            valid_moves.extend(self._get_moves_for_location(people_at_location, 'L'))
            valid_moves.extend(self._get_moves_for_location(people_at_location, 'R'))
            return valid_moves
        else:  # state.boat == 'R'
            # Boat is at right bank, can move 1 or 2 people to the island
            people_at_location = state.right
            destination = 'I'
        
        return self._get_moves_for_location(people_at_location, destination)
    
    def _get_moves_for_location(self, people_at_location: FrozenSet[str], destination: str) -> List[Tuple[FrozenSet[str], str]]:
        """Get all possible moves for a specific source location and destination."""
        valid_moves = []
        # One person in the boat
        for person in people_at_location:
            valid_moves.append((frozenset([person]), destination))
        
        # Two people in the boat
        people_list = list(people_at_location)
        for i in range(len(people_list)):
            for j in range(i+1, len(people_list)):
                valid_moves.append((frozenset([people_list[i], people_list[j]]), destination))
        
        return valid_moves
    
    def get_next_states(self, state: JealousHusbandsState) -> List[JealousHusbandsState]:
        """Generate all valid next states from the current state."""
        next_states = []
        
        for people_moving, destination in self.get_valid_moves(state):
            # Create new sets for the next state
            new_left = state.left.copy()
            new_island = state.island.copy()
            new_right = state.right.copy()
            
            # Remove people from current location
            if state.boat == 'L':
                new_left = frozenset(new_left - people_moving)
            elif state.boat == 'I':
                new_island = frozenset(new_island - people_moving)
            else:  # state.boat == 'R'
                new_right = frozenset(new_right - people_moving)
            
            # Add people to new location
            if destination == 'L':
                new_left = frozenset(new_left | people_moving)
            elif destination == 'I':
                new_island = frozenset(new_island | people_moving)
            else:  # destination == 'R'
                new_right = frozenset(new_right | people_moving)
            
            new_state = JealousHusbandsState(new_left, new_island, new_right, destination)
            
            # Only add valid states
            if self.is_valid_state(new_state):
                next_states.append(new_state)
        
        return next_states
    
    def dfs_search(self) -> List[JealousHusbandsState]:
        """Solve the problem using Depth-First Search."""
        start_time = time.time()
        
        stack = [(self.initial_state, [self.initial_state])]
        visited = set()
        
        nodes_expanded = 0
        max_fringe_size = 1
        
        while stack:
            current_state, path = stack.pop()
            nodes_expanded += 1
            
            if current_state in visited:
                continue
                
            visited.add(current_state)
            
            if self.goal_state_check(current_state):
                end_time = time.time()
                print(f"DFS Statistics:")
                print(f"  Time taken: {end_time - start_time:.4f} seconds")
                print(f"  Nodes expanded: {nodes_expanded}")
                print(f"  Max fringe size: {max_fringe_size}")
                print(f"  Path length: {len(path)}")
                return path
            
            next_states = self.get_next_states(current_state)
            
            # Add states to stack in reverse order so we explore left to right
            for next_state in reversed(next_states):
                if next_state not in visited:
                    stack.append((next_state, path + [next_state]))
            
            max_fringe_size = max(max_fringe_size, len(stack))
        
        print("No solution found with DFS")
        return []
    
    def backtracking_search(self) -> List[JealousHusbandsState]:
        """Solve the problem using Backtracking."""
        start_time = time.time()
        
        visited = set()
        path = []
        nodes_expanded = 0
        
        def backtrack(state: JealousHusbandsState) -> bool:
            nonlocal nodes_expanded
            nodes_expanded += 1
            
            if self.goal_state_check(state):
                path.append(state)
                return True
            
            if state in visited:
                return False
                
            visited.add(state)
            path.append(state)
            
            for next_state in self.get_next_states(state):
                if next_state not in visited:
                    if backtrack(next_state):
                        return True
            
            # Backtrack if no solution found from this state
            path.pop()
            return False
        
        result = backtrack(self.initial_state)
        
        end_time = time.time()
        print(f"Backtracking Statistics:")
        print(f"  Time taken: {end_time - start_time:.4f} seconds")
        print(f"  Nodes expanded: {nodes_expanded}")
        print(f"  Path length: {len(path)}")
        
        if result:
            return path
        else:
            print("No solution found with Backtracking")
            return []
    
    def heuristic(self, state: JealousHusbandsState) -> int:
        """Heuristic for A* search: number of people not yet at the right bank."""
        return len(self.all_people) - len(state.right)
    
    def a_star_search(self) -> List[JealousHusbandsState]:
        """Solve the problem using A* search."""
        start_time = time.time()
        
        # Priority queue entries are (f-value, move_number, state, path)
        # move_number is used to break ties consistently
        frontier = [(self.heuristic(self.initial_state), 0, self.initial_state, [self.initial_state])]
        visited = set()
        
        nodes_expanded = 0
        max_fringe_size = 1
        move_counter = 1
        
        while frontier:
            _, _, current_state, path = heapq.heappop(frontier)
            
            if current_state in visited:
                continue
                
            nodes_expanded += 1
            visited.add(current_state)
            
            if self.goal_state_check(current_state):
                end_time = time.time()
                print(f"A* Statistics:")
                print(f"  Time taken: {end_time - start_time:.4f} seconds")
                print(f"  Nodes expanded: {nodes_expanded}")
                print(f"  Max fringe size: {max_fringe_size}")
                print(f"  Path length: {len(path)}")
                return path
            
            for next_state in self.get_next_states(current_state):
                if next_state not in visited:
                    g_value = len(path)  # cost so far is the path length
                    h_value = self.heuristic(next_state)
                    f_value = g_value + h_value
                    
                    heapq.heappush(frontier, (f_value, move_counter, next_state, path + [next_state]))
                    move_counter += 1
            
            max_fringe_size = max(max_fringe_size, len(frontier))
        
        print("No solution found with A*")
        return []


def print_solution_path(path: List[JealousHusbandsState]):
    """Print the solution path in a readable format."""
    print("\nSolution Path:")
    for i, state in enumerate(path):
        print(f"Step {i}:")
        print(f"  Left Bank: {sorted(state.left)}")
        print(f"  Island: {sorted(state.island)}")
        print(f"  Right Bank: {sorted(state.right)}")
        print(f"  Boat at: {state.boat}")
        print()


# Example usage with 2 couples
if __name__ == "__main__":
    n = 3  # Number of couples
    problem = JealousHusbandsProblem(n)
    
    print(f"Solving Jealous Husbands Problem with {n} couples and an island...")
    
    print("\n=== DFS Solution ===")
    dfs_path = problem.dfs_search()
    print_solution_path(dfs_path)
    
    print("\n=== Backtracking Solution ===")
    backtracking_path = problem.backtracking_search()
    print_solution_path(backtracking_path)
    
    print("\n=== A* Solution ===")
    a_star_path = problem.a_star_search()
    print_solution_path(a_star_path)

Solving Jealous Husbands Problem with 3 couples and an island...

=== DFS Solution ===
DFS Statistics:
  Time taken: 0.0190 seconds
  Nodes expanded: 334
  Max fringe size: 307
  Path length: 203

Solution Path:
Step 0:
  Left Bank: ['H1', 'H2', 'H3', 'W1', 'W2', 'W3']
  Island: []
  Right Bank: []
  Boat at: L

Step 1:
  Left Bank: ['H1', 'H3', 'W1', 'W3']
  Island: ['H2', 'W2']
  Right Bank: []
  Boat at: I

Step 2:
  Left Bank: ['H1', 'H2', 'H3', 'W1', 'W3']
  Island: ['W2']
  Right Bank: []
  Boat at: L

Step 3:
  Left Bank: ['H1', 'H2', 'H3', 'W3']
  Island: ['W1', 'W2']
  Right Bank: []
  Boat at: I

Step 4:
  Left Bank: ['H1', 'H2', 'H3', 'W2', 'W3']
  Island: ['W1']
  Right Bank: []
  Boat at: L

Step 5:
  Left Bank: ['H1', 'H2', 'H3', 'W2']
  Island: ['W1', 'W3']
  Right Bank: []
  Boat at: I

Step 6:
  Left Bank: ['H1', 'H2', 'H3', 'W1', 'W2']
  Island: ['W3']
  Right Bank: []
  Boat at: L

Step 7:
  Left Bank: ['H1', 'H2', 'H3', 'W1']
  Island: ['W2', 'W3']
  Right Bank: []
